Langsmith

In [109]:

%pip install -qU langsmith

Note: you may need to restart the kernel to use updated packages.


In [110]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
     

import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
     


In [111]:


%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


Semantic Model Extraction

In [112]:
%pip install -q langchain-community langchain-openai langchain_experimental neo4j
     

Note: you may need to restart the kernel to use updated packages.


In [113]:
from langchain.graphs import Neo4jGraph

url = "bolt://localhost:7687"
username = "neo4j"
password = "12345678"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [114]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
     

     


Graph QA using GraphCypherQAChain

In [115]:
%pip install  --quiet langchain langchain-openai langchain-community neo4j
     

Note: you may need to restart the kernel to use updated packages.


In [116]:
from langchain.graphs import Neo4jGraph
url = "bolt://localhost:7687"
username = "neo4j"
password = "12345678"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)
    

In [117]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
     

from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
import os

chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-4o-mini"), # gpt-4o-mini	gpt-3.5-turbo
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    verbose=True,
    allow_dangerous_requests=True
)

In [119]:

import gradio as gr
# Defining questions
questions = {
    "question_1": "What is the sensor type of Sentinel-2 L2A Maja Products",
    "question_2": "What is the spatial extent of Sentinel-2 L2A Maja Products?",
    "question_3": "What is the spatial extent of Sentinel-1 Floodmasks (Data4Human)?",
    "question_4": "Give me the download links for Sentinel-2 L2A Maja Products?",
    "question_5": "What is the spatial extent of S5P Tropomi L4 P1D NO2surf?",
    "question_6": "What is the reference system used for Sentinel-2 L2A Maja Products?",
    "question_7": "What is the ISO standards for Sentinel-2 L2A Maja Products?",
    "question_8": "what is the temporal extent of S5P Tropomi L3 P1D CF?",
    "question_9": "what is the orbit type of Sentinel-2 L2A Maja Products?",
    "question_10": "provide the geojson of Sentinel-2 L2A Maja Products?",
    "question_11": "provide the geojson of Global WaterPack - MODIS?",
    "question_12": "what is the visualisation link of EnMAP L2A HSI Products?",
    "question_13": "What id the sensor type of S5P Tropomi L3 P1D O3?",
    "question_14": "what is the visualization link of Sentinel-2 L3A Monthly WASP Products?",
    
   
}

def get_answer(messages):
    question = messages[-1][0] 
    question_key = next((key for key, value in questions.items() if value.lower() == question.lower()), None)
    try:
        if question_key:
            response = chain.invoke(questions[question_key])
        else:
            response = chain.invoke(question)
        
        if isinstance(response, dict) and 'result' in response:
            return messages[:-1] + [[question, response['result'].strip()]] 
        else:
            return messages[:-1] + [[question, "No result found or invalid response format."]]
    except Exception as e:
        return messages[:-1] + [[question, f"An error occurred while processing your question: {str(e)}"]]

def print_like_dislike(data: gr.LikeData):
    print(f"Message index: {data.index}, Liked: {data.liked}, Content: {data.value}")

with gr.Blocks() as iface:
    gr.Markdown("# STAChat")
    chatbot = gr.Chatbot(label="Chat History", avatar_images=(
            None,
            "bot.png",))
    user_input = gr.Textbox(placeholder="Ask your question here...", label="Your Question")
    submit_button = gr.Button("Send")
    
    def chat_interaction(messages, user_query):
        if user_query:
            messages = messages + [[user_query, None]]
            return get_answer(messages), ""  
        return messages, ""

    chat_msg = user_input.submit(chat_interaction, [chatbot, user_input], [chatbot, user_input])
    chat_msg.then(lambda: gr.Textbox(interactive=True), None, [user_input])

    chatbot.like(print_like_dislike, None, None, like_user_message=True)

    submit_button.click(
        chat_interaction, 
        inputs=[chatbot, user_input], 
        outputs=[chatbot, user_input]
    )
    user_input.submit(
        chat_interaction, 
        inputs=[chatbot, user_input], 
        outputs=[chatbot, user_input]
    )

if __name__ == '__main__':
    iface.launch()

     

/Users/iprincetech/Desktop/Question-Answering-System-for-Raster-Data/.venv/lib/python3.13/site-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.
